## `siamese network implementation`
- takes 2 images and tries to compare features of 2 images, if features are matching mark it as similar else not similar.
- our mobile use **facemesh algo(detect landmarks) + siamese(classify) + mobilenet** network in image recognition

In [79]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
import random
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten,Dense,Dropout,Lambda,Input
from tensorflow.keras.layers import Conv2D,Activation,AveragePooling2D

In [80]:
from tensorflow.keras.datasets import mnist

In [81]:
from tensorflow.keras import backend as k

In [82]:
num_classes = 10
def euclidean_dis(vects):
    x,y = vects
    sum_square = k.sum(k.square(x-y),axis=1,keepdims=True)
    return k.sqrt(k.maximum(sum_square,k.epsilon()))

In [83]:
def contrastive_loss(y_true,y_pred):
    y_true = tf.dtypes.cast(y_true,tf.float32)
    y_pred = tf.dtypes.cast(y_pred,tf.float32)
    margin = 1
    square_pred = k.square(y_pred)
    margin_square = k.square(k.maximum(margin - y_pred,0))
    return k.mean(y_true*square_pred+(1-y_true) * margin_square)

In [84]:
def create_pairs(x,digit_indices):
    pairs = []
    labels = []
    n = min([len(digit_indices[d]) for d in range(num_classes)]) - 1
    for d in range(num_classes):
        for i in range(n):
            z1,z2 = digit_indices[d][i],digit_indices[d][i+1] # static number
            pairs += [[x[z1],x[z2]]]
            inc = random.randrange(1,num_classes)
            dn = (d + inc) % num_classes
            z1,z2 = digit_indices[d][i],digit_indices[dn][i] # random number
            pairs += [[x[z1],x[z2]]]
            labels += [1,0]
    return np.array(pairs),np.array(labels)

### `neural network in a function way`*****

In [85]:
def create_neural_net(input_shape):
    input = Input(shape=input_shape)
    x = Conv2D(4,(5,5),activation='tanh')(input)
    x = AveragePooling2D(pool_size=(2,2))(x)
    x = Conv2D(16,(5,5),activation='tanh')(x)
    x = AveragePooling2D(pool_size=(2,2))(x)
    x = Flatten()(x)
    x = Dense(10,activation='tanh')(x)
    model = Model(input,x)
    model.summary()
    return model

In [86]:
# Conv2D??

In [87]:
def euclidean_dist_output_shape(shapes):
    shape1,shape2 = shapes
    return (shape1[0],1)

################ Both are same methods ####################

def compute_accuracy(y_true,y_pred):
    pred = y_pred.ravel() < 0.5
    return np.mean(pred == y_true)

def accuracy(y_true,y_pred):
    return k.mean(k.equal(y_true,k.cast(y_pred < 0.5,y_true.dtype)))

### `Create data`

In [88]:
(X_train,y_train),(X_test,y_test) = mnist.load_data()

In [89]:
X_train.shape,X_test.shape

((60000, 28, 28), (10000, 28, 28))

In [90]:
X_train = X_train.reshape(X_train.shape[0],28,28,1)
X_test = X_test.reshape(X_test.shape[0],28,28,1)
input_shape = (1,28,28)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

# normalizing
X_train /= 255  
X_test /= 255

(60000, 28, 28, 1)
(10000, 28, 28, 1)
(60000,)
(10000,)


In [91]:
input_shape = X_train.shape[1:]
input_shape = (28,28,1)

In [92]:
digit_indices = [np.where(y_train == i)[0] for i in range(num_classes)]
tr_pairs,tr_y = create_pairs(X_train,digit_indices)

digit_indices = [np.where(y_test == i)[0] for i in range(num_classes)]
te_pairs,te_y = create_pairs(X_test,digit_indices)

base_network = create_neural_net(input_shape)

Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 24, 24, 4)         104       
_________________________________________________________________
average_pooling2d_8 (Average (None, 12, 12, 4)         0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 8, 8, 16)          1616      
_________________________________________________________________
average_pooling2d_9 (Average (None, 4, 4, 16)          0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                2570

In [93]:
tr_pairs.shape

(108400, 2, 28, 28, 1)

In [94]:
tr_y.shape

(108400,)

In [95]:
te_pairs.shape

(17820, 2, 28, 28, 1)

In [96]:
te_y.shape

(17820,)

In [97]:
input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)
processed_a = base_network(input_a)
processed_b = base_network(input_b)

distance = Lambda(euclidean_dis,
                  output_shape=euclidean_dist_output_shape)([processed_a,processed_b])
model = Model([input_a,input_b],distance)

In [98]:

model.compile(loss=contrastive_loss,optimizer='adam',metrics=['accuracy'])

model.fit([tr_pairs[:,0],tr_pairs[:,1]],
          tr_y,
          batch_size=128,
          epochs=20,
          validation_data=([te_pairs[:,0],te_pairs[:,1]],te_y))

Epoch 1/20
847/847 [==============================] - 64s 72ms/step - loss: 0.1085 - accuracy: 0.1207 - val_loss: 0.0764 - val_accuracy: 0.0732
Epoch 2/20
847/847 [==============================] - 66s 78ms/step - loss: 0.0752 - accuracy: 0.0647 - val_loss: 0.0647 - val_accuracy: 0.0517
Epoch 3/20
847/847 [==============================] - 59s 69ms/step - loss: 0.0657 - accuracy: 0.0526 - val_loss: 0.0571 - val_accuracy: 0.0463
Epoch 4/20
847/847 [==============================] - 60s 71ms/step - loss: 0.0595 - accuracy: 0.0458 - val_loss: 0.0543 - val_accuracy: 0.0342
Epoch 5/20
847/847 [==============================] - 53s 62ms/step - loss: 0.0552 - accuracy: 0.0408 - val_loss: 0.0489 - val_accuracy: 0.0316
Epoch 6/20
847/847 [==============================] - 48s 57ms/step - loss: 0.0522 - accuracy: 0.0375 - val_loss: 0.0471 - val_accuracy: 0.0331
Epoch 7/20
847/847 [==============================] - 50s 59ms/step - loss: 0.0497 - accuracy: 0.0351 - val_loss: 0.0454 - val_accuracy:

### `prediction`

In [100]:
y_pred = model.predict([tr_pairs[:,0],tr_pairs[:,1]])
tr_acc = compute_accuracy(tr_y,y_pred)
y_pred = model.predict([te_pairs[:,0],te_pairs[:,1]])
te_acc = compute_accuracy(te_y,y_pred)
print('Accuracy on training data',(tr_acc*100))
print('Accuracy on testing data',(te_acc*100))

Accuracy on training data 97.39206642066421
Accuracy on testing data 97.46352413019079


### `plotting`

## `END ------------------------------------------------`